In [1]:
import json
from PIL import Image
from aoa_evaluate import LG_Dataset, build_dataloader, build_aspect_ratio_grouped_sampler, collate_batch
from tqdm import tqdm
import torch

In [3]:
with open('/nfs/turbo/coe-chaijy/jiayipan/Vision-Language-Project/EMNLP/Evaluation/dataset/30-Ana-VG_Dataset.json', 'r') as f:
    data = json.load(f)

dataset = LG_Dataset(data, "/nfs/turbo/coe-chaijy/datasets/flickr30k/flickr30k-images/", "/nfs/turbo/coe-chaijy/datasets/mscoco-2014/all/", "/scratch/chaijy_root/chaijy2/jiayipan/datasets/GQA/images/")

In [4]:
data['tests'][0]

{'pair_id': 0, 'mask_regions': [[1, 'banner', 'NOUN', [2, 8]]]}

In [5]:
dataloader = build_dataloader(dataset, 32, num_workers=4)
Data_Iter = iter(dataloader)

In [6]:
dataset[233]

{'img': tensor([[[-0.3541, -0.3541, -0.3541,  ...,  2.2489,  2.2489,  2.2489],
          [-0.3369, -0.3541, -0.3541,  ...,  2.2489,  2.2489,  2.2489],
          [-0.3027, -0.3369, -0.3712,  ...,  2.2489,  2.2489,  2.2489],
          ...,
          [-0.3369, -0.3369, -0.3541,  ..., -1.0219, -1.0390, -1.0390],
          [-0.3712, -0.3541, -0.3027,  ..., -0.9192, -0.9363, -0.9363],
          [-0.3883, -0.3541, -0.2856,  ..., -0.8849, -0.9020, -0.9020]],
 
         [[-0.0399, -0.0049,  0.0651,  ...,  2.4286,  2.4286,  2.4286],
          [-0.0224, -0.0049,  0.0476,  ...,  2.4286,  2.4286,  2.4286],
          [ 0.0126,  0.0126,  0.0301,  ...,  2.4286,  2.4286,  2.4286],
          ...,
          [-0.8452, -0.8277, -0.7927,  ..., -1.2829, -1.3004, -1.3179],
          [-0.9153, -0.8803, -0.7752,  ..., -1.1954, -1.2129, -1.2304],
          [-0.9503, -0.8978, -0.7577,  ..., -1.1604, -1.1779, -1.1954]],
 
         [[-0.8284, -0.8110, -0.7761,  ...,  2.6400,  2.6400,  2.6400],
          [-0.8110, -

In [7]:
s = build_aspect_ratio_grouped_sampler(dataset, 32)

In [8]:
indices = range(len(dataset))
aspect_ratios = []
for i in indices:
    height, width = dataset.get_height_and_width(i)
    aspect_ratio = float(width) / float(height)
    aspect_ratios.append(aspect_ratio)

In [9]:
from copy import deepcopy
O = None
A = None
B = None
for idx, t in enumerate(tqdm(s)):
    ratio_set = set()
    for i in t:
        ratio_set.add(aspect_ratios[i])
    if idx == 3545:
        O = deepcopy(t)
    if idx == 3546:
        A = deepcopy(t)
    if idx == 3547:
        B = deepcopy(t)
        break
    assert len(ratio_set) == 1

2311it [00:00, 71816.76it/s]


In [10]:
data_batch = [dataset[i] for i in A]
collate_batch(data_batch)

TypeError: 'NoneType' object is not iterable

In [21]:
# for dp_id in A:
#     this_cache_hw = dataset.get_height_and_width(dp_id)
#     this_cache_ratio = float(this_cache_hw[0]) / float(this_cache_hw[1])
#     this_hw = dataset[dp_id]['text_info']['img_size']
#     this_ratio = float(this_hw[1]) / float(this_hw[0])
#     diff = abs(this_cache_ratio - this_ratio)
#     print(diff)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.16400000000000003
0.16400000000000003
0.16400000000000003


In [11]:
ERROR_IDX = []
for dp_id, dp in enumerate(tqdm(dataset)):
    this_cache_hw = dataset.get_height_and_width(dp_id)
    this_cache_ratio = float(this_cache_hw[0]) / float(this_cache_hw[1])
    this_hw = dataset[dp_id]['text_info']['img_size']
    this_ratio = float(this_hw[1]) / float(this_hw[0])
    diff = abs(this_cache_ratio - this_ratio)
    if diff > 0.0001:
        ERROR_IDX.append(dp_id)
        print(dp_id)
torch.save(ERROR_IDX, 'ERROR_IDX.pth')

 99%|████████████████████████████████████▊| 54855/55176 [52:09<00:20, 15.43it/s]

54852
54853


100%|█████████████████████████████████████| 55176/55176 [52:26<00:00, 17.54it/s]


In [5]:
ERROR_IDX

[124297,
 124298,
 124299,
 124300,
 124301,
 124443,
 124444,
 125951,
 126419,
 152423,
 152424,
 152425,
 152426,
 152702,
 152703,
 152940,
 152941,
 152942,
 153481,
 153482,
 153483,
 153484]

In [15]:
len(data['tests'])

155751

In [9]:
Ts = [data['tests'][i] for i in ERROR_IDX]

In [17]:
for i in sorted(ERROR_IDX, reverse=True):
    del data['tests'][i]

In [19]:
with open('/nfs/turbo/coe-chaijy/jiayipan/Vision-Language-Project/EMNLP/Evaluation/dataset/Fixed_Ana_VG_Dataset.json', 'w') as f:
    json.dump(data,f)

In [18]:
len(data['tests'])

155729